<a href="https://colab.research.google.com/github/fernandaleonn/PythonCourse/blob/main/2023_11_15_decoradores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Decoradores
===========

**Date:** 2023-11-15



## Idea



Los *decorators* son un caso particular de funciones que aceptan funciones y regresan funciones. Por ejemplo:



In [8]:
def saludo_func(func):
    def salida():
        print("Voy a ejecutar una función")
        func()
        print("Ya terminé")
    return salida

def hello():
    print("Hello world")

def hola():
    print("Hola mundo")


#hello()
#hello2 = saludo_func(hello)
#hello2()
hola = saludo_func(hola)
hola()

Voy a ejecutar una función
Hola mundo
Ya terminé


In [3]:
def execute_twice(func):
    def twice():
        func()
        func()
    return twice

hello3 = execute_twice(hello)
hello3()

Hello world
Hello world


Un resultado parecido se obtiene "decorando" la función.



In [ ]:
@saludo_func
def hello():
    print("Hello world")

@saludo_func
def hola():
    print("Hola mundo")

hola()

In [ ]:
@execute_twice
def hola():
    print("Hola mundo")

hola()

Si queremos decorar una función con argumentos, así como lo tenemos definido, no funciona, pues tenemos que pasar los argumentos de la función decorada. Como queremos que el decorador sirva para cualquier cantidad y tipo de argumentos, usaremos la sintaxis que nos permite un número arbitrario de argumentos.



In [ ]:
@saludo_func
def suma(x, y):
    print(f"La suma es {x+y}")

suma(2, 3)

Como ejemplo de esa sintaxis:



In [ ]:
def muchos_argumentos(*args):
    # print(type(args))
    return args[0]

muchos_argumentos(1, 2, 3), muchos_argumentos("hola", "adios")

In [ ]:
def argumentos_keyword(**kwargs):
    print(type(kwargs))
    print(kwargs)
    return kwargs['inicio']

argumentos_keyword(inicio="hola", final="adios", cantidad=3)

In [ ]:
def saludo_func(func):
    def salida(*args, **kwargs):
        print("Voy a ejecutar una función")
        func(*args, **kwargs)
        print("Ya terminé")
    return salida

@saludo_func
def suma(x, y):
    print(f"La suma es {x+y}")

suma(2, 3)

**TAREA** Hacer una nueva versión de la función `suma` que acepte una cantidad arbitraria de argumentos numéricos y regrese la suma. A continuación, decorar esa nueva versión con `saludo_func`.



In [ ]:
def saludos(veces, saludo="Hola"):
    for i in range(veces):
        print(saludo)

saludos(4)

In [ ]:
@saludo_func
def saludos(veces, saludo="Hola"):
    for i in range(veces):
        print(saludo)

saludos(3, saludo="Adiós")

En los ejemplos anteriores hemos utilizado funciones con *efectos secundarios* (como *print*). Supongamos que queremos decorar una función que regrese algo.



In [ ]:
def ejecuta_doble(func):
    def decorador(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return decorador

@ejecuta_doble
def suma(x, y):
    print("El resultado es:")
    return x+y

suma(2, 3)

Para que funcione, el decorador debe de regresar el resultado de la función decorada.



In [ ]:
def ejecuta_doble(func):
    def decorador(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return decorador

@ejecuta_doble
def suma(x, y):
    print("El resultado es:")
    return x+y

a = suma(2, 3)

In [ ]:
a

## Decoradores predefinidos



### property



Recordemos la clase que definimos para números complejos:



In [ ]:
from math import sqrt

class NúmeroComplejo:
    def __init__(self, parte_real, parte_imaginaria):
        self.r = parte_real
        self.i = parte_imaginaria

    def módulo(self):
        return sqrt(self.r**2 + self.i**2)

z = NúmeroComplejo(3, -1)
z.r, z.i, z.módulo()

En este caso, `módulo` es en realidad un atributo del número complejo, pero como está calculado por una función, se deben incluir los paréntesis. Pero decorando el método con `@property` (que ya viene definido con Python), se puede convertir en atributo.



In [ ]:
from math import sqrt

class NúmeroComplejo:
    def __init__(self, parte_real, parte_imaginaria):
        self.r = parte_real
        self.i = parte_imaginaria

    @property
    def módulo(self):
        return sqrt(self.r**2 + self.i**2)

z = NúmeroComplejo(3, -1)
z.r, z.i, z.módulo

### cache



In [ ]:
from functools import cache

# @cache
def factorial(n):
    return n * factorial(n-1) if n else 1

%time factorial(1000)

In [ ]:
%time factorial(1001)

### dataclass



También se pueden decorar clases. Por ejemplo, `@dataclass` puede definir de manera rápida los métodos `__init__` y `__repr__` en clases sencillas. (A partir de Python 3.7)



In [ ]:
from dataclasses import dataclass

from math import sqrt

@dataclass
class NúmeroComplejo:
    r: float
    i: float

    @property
    def módulo(self):
        return sqrt(self.r**2 + self.i**2)

z = NúmeroComplejo(3, -1)
z.r, z.i, z.módulo, z

### wraps



Una función "conoce" su nombre y su documentación.



In [ ]:
def suma(x, y):
    """Regresa una suma"""
    print(f"La suma es {x+y}")

suma.__name__, suma.__doc__

Pero al ser decorada, ya no lo recuerda exactamente, sino que reporta el nombre del decorador.



In [ ]:
def ejecuta_doble(func):
    def decorador(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return decorador

@ejecuta_doble
def suma(x, y):
    """Regresa una suma"""
    print(f"La suma es {x+y}")

suma.__name__, suma.__doc__

Entonces, los decoradores deben también ser decorados. Este es un caso de un decorador con argumentos, su argumento es la función a decorar.



In [ ]:
from functools import wraps

def ejecuta_doble(func):
    @wraps(func)
    def decorador(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return decorador

@ejecuta_doble
def suma(x, y):
    """Regresa una suma"""
    print(f"La suma es {x+y}")

suma.__name__, suma.__doc__

Por eso, en la página [https://realpython.com/primer-on-python-decorators/>](https://realpython.com/primer-on-python-decorators/>)recomiendan que todos los decoradores sigan este patrón:



In [ ]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

### staticmethod and classmethod



Estos ejemplos son tomados de [https://aiflavours.com/python-decorator/](https://aiflavours.com/python-decorator/)

Consideremos una clase que tiene un método independiente de `self`, el cual quisiéramos usar en una instancia de la clase.



In [ ]:
class Car:
    def __init__(self, colour):
        self.colour = colour

    def get_number_of_wheels():
        return 4

red_car = Car(colour="red")

# ok
Car.get_number_of_wheels()

# not ok
# red_car.get_number_of_wheels(),

In [ ]:
class Car:
    def __init__(self, colour):
        self.colour = colour

    @staticmethod
    def get_number_of_wheels():
        return 4

red_car = Car(colour="red")
red_car.get_number_of_wheels(), Car.get_number_of_wheels()

Por otro lado, podemos usar `@classmethod` para crear instancias de una clase usando un método de la misma clase.



In [ ]:
class Car:
    def __init__(self, colour):
        self.colour = colour

    @classmethod
    def load(cls, colour):
        return cls(colour)

blue_car = Car.load("blue")

type(blue_car), blue_car.colour